<table style='border: none' align='left'>
   <tr style='border: none'>
      <th style='border: none'><font face='verdana' size='5' color='black'><b>Iris multi-class classification with scikit-learn</b></th>
      <th style='border: none'><img src='https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true' alt='Watson Machine Learning icon' height='40' width='40'></th>
   </tr>
   <tr style='border: none'>
       <th style='border: none'><img src='https://cloud.ibm.com/docs-content/v1/content/6b4d680b3820820b97fbf9b7db3c9d3658db8ac2/tutorials/images/solution22-build-machine-learning-model/iris_machinelearning.png' width='600' alt='Icon'> </th>
   </tr>
</table>

**Note:** _The notebook is part of a solution tutorial [Build, deploy, test, retrain and monitor a predictive machine learning model](https://cloud.ibm.com/docs/tutorials?topic=solution-tutorials-create-deploy-retrain-machine-learning-model) and used for demonstrating IBM Watson OpenScale only._

This notebook utilizes <a href="https://pypi.python.org/pypi/watson-machine-learning-client" target="_blank" rel="noopener noreferrer">watson-machine-learning-client</a> package in order to save, deploy, and score a predictive model.

Some familiarity with Python is helpful. This notebook uses Python 3.6, scikit-learn, and the Watson Machine Learning (WML) API client (`watson-machine-learning-client`).

You will use the sample **iris flower data set**.The Iris dataset was used in R.A. Fisher's classic 1936 paper, [The Use of Multiple Measurements in Taxonomic Problems](http://rcs.chemometrics.ru/Tutorials/classification/Fisher.pdf), and can also be found on the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/). This small dataset is often used for testing out machine learning algorithms and visualizations. The aim is to classify Iris flowers among three species (Setosa, Versicolor or Virginica) from measurements of length and width of sepals and petals. The iris data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant.

<a id='setup'></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>)

- Configure your local python environment:
  + python 3.6
  + scikit-learn 0.20.0
  + watson-machine-learning-client

### Provide WML credentials
**Tip**: Authentication information (your credentials) can be found in the <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-get-wml-credentials.html" target="_blank" rel="noopener no referrer">Service credentials</a> tab of the service instance that you created on the IBM Cloud. <BR>If you cannot find the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials here without any trailing spaces.

In [ ]:
wml_credentials = {
    "apikey": "<API_KEY>",
    "instance_id": "<INSTANCE_ID>",
    "url": "<URL>"
}

**Tip**: Run the cell below to install packages from <a href='https://pypi.python.org/pypi' target='_blank' rel='noopener no referrer'>PyPI</a>.

In [ ]:
# Remove current wml package files.
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [ ]:
# Install watson-machine-learning-client required for online deployment and scoring.
!pip install --user watson-machine-learning-client --upgrade

<a id='load'></a>
## Load and explore data

In this section, you will load the data from a sample iris `.csv` file

In [ ]:
!pip install scikit-learn==0.20.0

In [ ]:
import pandas as pd

iris = pd.read_csv("https://raw.githubusercontent.com/IBM-Cloud/ml-iris-classification/master/data/iris.csv")
# download the file
print(iris.shape)
print(iris.columns)

The sample data set consists of 8x8 pixel images of hand-written digits.

In [ ]:
iris.shape

Display the first digit data and label using **data** and **target**. The hand-written digit shown below is an 8x8 pixel image represented by an array.

In [ ]:
iris.describe

<a id='model'></a>
## Create a scikit-learn model

### Split data<a id='prep'></a>

In this section you will split your data into: 
- Train data set
- Test data set

In [ ]:
from sklearn.model_selection import train_test_split
iris_training, iris_test = train_test_split(iris, test_size=0.2)
# separate x and y variables from data
iris_training_y = iris_training["species"]
iris_training_x = iris_training.drop("species", axis=1)

iris_test_y = iris_test["species"]
iris_test_x = iris_test.drop("species", axis=1)

In [ ]:
print(iris_training_x.count())

### Train a model<a id='train'></a>

In [ ]:
from sklearn import tree
classifier=tree.DecisionTreeClassifier()

In [ ]:
classifier.fit(iris_training_x,iris_training_y)

### Test the model

In [ ]:
predictions=classifier.predict(iris_test_x)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(iris_test_y,predictions))

**Note:** You can tune your model, or try creating classification models to achieve better accuracy. For simplicity of this example, the tuning section and model selection are omitted.

<a id='persistence'></a>
## Save, load, and delete a model in the WML repository

In this section, you will learn how to use the `watson-machine-learning-client` package to manage your model in the WML repository.

**Tip**: You can find more information about the watson-machine-learning-client <a href="https://console.bluemix.net/docs/services/service_credentials.html#service_credentials" target="_blank" rel="noopener no referrer">here</a>[](https://wml-api-pyclient.mybluemix.net).

### Set up the WML instance<a id='work'></a>

First, import required modules.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**Note**: A deprecation warning is returned from scikit-learn package that does not impact watson machine learning client functionalities.
Authenticate to the Watson Machine Learning service on the IBM Cloud.

#### Instantiate the WML API client object. 

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

#### Get instance details.

In [ ]:
import json

instance_details = client.service_instance.get_details()

### Save the model in the WML repository<a id='save'></a>

Define the model name when saving the model in the WML repository.

In [ ]:
published_model = client.repository.store_model(model=classifier, meta_props={'name':'scikit iris model'}, \
                                                training_data=iris_training_x, training_target=iris_training_y)

#### Get information about a specific model in the WML repository.

In [ ]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

print(json.dumps(model_details, indent=2))

#### Get information of all the models in the WML repository.

In [ ]:
models_details = client.repository.list_models()

### Load a model from the WML repository<a id='load_model'></a>

In this subsection, you will learn how to load a saved model from a specific WML instance.

In [ ]:
loaded_model = client.repository.load(published_model_uid)

Make test predictions to check that the model has been loaded correctly.

In [ ]:
test_predictions = loaded_model.predict(iris_test_x[:10])
print(iris_test_x[:10])

In [ ]:
# The predictions made by the model.
print(test_predictions)

As you can see, you are able to make predictions which means that the model has loaded correctly. You have now learned how save and load the model in/from the WML repository.

### Delete a model from the WML repository<a id='delete'></a>

The code in the following cell deletes a published model from the WML repository. The code is not executable at this stage because you still need the model for deployment.

<a id='scoring'></a>
## Deploy and score data in the WML repository

In this section, you will learn how to use the WML API client to perform online scoring and score a test data record.

### Create an online deployment for the published model<a id='create'></a>

In [ ]:
created_deployment = client.deployments.create(published_model_uid, 'Deployment of iris model')

**Note**: You use the deployment url saved in published_model_uid object. The next section shows you how to retrieve the deployment url from the WML instance.

Now, you can define and print an online scoring endpoint. 

 5.2 Get deployments<a id='getdeploy'></a>

In [ ]:
deployments = client.deployments.get_details()
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

You can get the deployment_url by parsing the deployment details for the last deployed model.

### Score data<a id='score'></a>

Use the following method to run a test scoring request against the deployed model.

**Action**: Prepare scoring payload with the records to score.

In [ ]:
import random
for i in range(100):
    array_of_values_to_be_scored = [round(random.uniform(0.0, 10.0), 1), round(random.uniform(
        0.0, 10.0), 1), round(random.uniform(0.0, 10.0), 1), round(random.uniform(0.0, 10.0), 1)]
    another_array_of_values_to_be_scored = [round(random.uniform(0.0, 10.0), 1), round(random.uniform(
        0.0, 10.0), 1), round(random.uniform(0.0, 10.0), 1), round(random.uniform(0.0, 10.0), 1)]
    scoring_data = {
        'fields': ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], 
        'values': [array_of_values_to_be_scored,another_array_of_values_to_be_scored]
    }
    predictions = client.deployments.score(scoring_endpoint, scoring_data)

Use the ``client.deployments.score()`` method to run the scoring.

###  Delete the deployment<a id='deldeploy'></a>

Use the following method to delete the deployment.

You can check that your deployment was deleted by generating a list of your saved deployments:

### Delete the model<a id='delmodel'></a>

You can check that your model was deleted by generating a list of your saved models:

<a id='summary'></a>
## Summary and next steps     

<div style='background:#F5F7FA; height:110px; padding: 2em; font-size:14px;'>
<span style='font-size:18px;color:#152935;'>Love this notebook? </span>
<span style='font-size:15px;color:#152935;float:right;margin-right:40px;'>Don't have an account yet?</span><br>
<span style='color:#5A6872;'>Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style='border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;'><a href='https://ibm.co/wsnotebooks' target='_blank' style='color: #3d70b2;text-decoration: none;'>Sign Up</a></span><br>
</div>